In [1]:
using Flux, Flux.Data.MNIST
using Flux: onehotbatch, argmax, crossentropy, throttle
using Base.Iterators: repeated, partition
# using CuArrays

In [2]:
imgs = MNIST.images()
labels = onehotbatch(MNIST.labels(), 0:9)

10×60000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 false   true  false  false  false  …  false  false  false  false  false
 false  false  false   true  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false   true  false  false  false
 false  false   true  false  false     false  false  false  false  false
  true  false  false  false  false  …  false  false   true  false  false
 false  false  false  false  false     false  false  false   true  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false      true  false  false  false   true
 false  false  false  false   true     false  false  false  false  false

In [3]:
imgs[3]

#### Make partitions of the data (batches)

In [4]:

# Partition into batches of size 1,000
train = [(cat(4, float.(imgs[i])...), labels[:,i])
         for i in partition(1:60_000, 250)]

train = gpu.(train);

In [5]:
length(train)

240

In [6]:
typeof(imgs[3])

Array{ColorTypes.Gray{FixedPointNumbers.Normed{UInt8,8}},2}

In [7]:
size(train)

(240,)

In [8]:
size(train[1][1])

(28, 28, 1, 250)

In [9]:
size(train[1][2])

(10, 250)

In [10]:

# Prepare test set (first 1,000 images)
tX = cat(4, float.(MNIST.images(:test)[1:1000])...) |> gpu
tY = onehotbatch(MNIST.labels(:test)[1:1000], 0:9) |> gpu

m = Chain(
  Conv((2,2), 1=>32, relu),
  x -> maxpool(x, (2,2)),
  Conv((2,2), 32=>8, relu),
  x -> maxpool(x, (2,2)),
  x -> reshape(x, :, size(x, 4)),
  Dense(288, 10), softmax) |> gpu


Chain(Conv((2, 2), 1=>32, NNlib.relu), #3, Conv((2, 2), 32=>8, NNlib.relu), #4, #5, Dense(288, 10), NNlib.softmax)

In [11]:
size(tX), size(tY)

((28, 28, 1, 1000), (10, 1000))

#### Defining a loss function

In [12]:
loss(x, y) = crossentropy(m(x), y)

loss (generic function with 1 method)

#### Defining an evaluation metric

In [13]:
accuracy(x, y) = mean(argmax(m(x)) .== argmax(y))

accuracy (generic function with 1 method)

In [14]:
evalcb = throttle(() -> @show(accuracy(tX, tY)), 10)

(::throttled) (generic function with 1 method)

#### Define a an optimization procedure

In [15]:
length(train)

240

In [16]:
opt = ADAM(params(m))

(::#80) (generic function with 1 method)

In [17]:
Flux.train!(loss, train, opt, cb = evalcb)

accuracy(tX, tY) = 0.114
accuracy(tX, tY) = 0.1
accuracy(tX, tY) = 0.353
accuracy(tX, tY) = 0.64
accuracy(tX, tY) = 0.721
accuracy(tX, tY) = 0.77
accuracy(tX, tY) = 0.806
accuracy(tX, tY) = 0.822
accuracy(tX, tY) = 0.836
accuracy(tX, tY) = 0.836
accuracy(tX, tY) = 0.843
accuracy(tX, tY) = 0.857
accuracy(tX, tY) = 0.864


In [18]:
for i in 1:10
    Flux.train!(loss, train, opt, cb = evalcb)
end

accuracy(tX, tY) = 0.861
accuracy(tX, tY) = 0.865
accuracy(tX, tY) = 0.876
accuracy(tX, tY) = 0.873
accuracy(tX, tY) = 0.875
accuracy(tX, tY) = 0.888
accuracy(tX, tY) = 0.895
accuracy(tX, tY) = 0.899
accuracy(tX, tY) = 0.901
accuracy(tX, tY) = 0.901
accuracy(tX, tY) = 0.901
accuracy(tX, tY) = 0.901
accuracy(tX, tY) = 0.9
accuracy(tX, tY) = 0.909
accuracy(tX, tY) = 0.91
accuracy(tX, tY) = 0.91
accuracy(tX, tY) = 0.908
accuracy(tX, tY) = 0.911
accuracy(tX, tY) = 0.907
accuracy(tX, tY) = 0.915
accuracy(tX, tY) = 0.919
accuracy(tX, tY) = 0.914
accuracy(tX, tY) = 0.917
accuracy(tX, tY) = 0.922
accuracy(tX, tY) = 0.919
accuracy(tX, tY) = 0.915
accuracy(tX, tY) = 0.919
accuracy(tX, tY) = 0.923
accuracy(tX, tY) = 0.928
accuracy(tX, tY) = 0.929
accuracy(tX, tY) = 0.927
accuracy(tX, tY) = 0.926
accuracy(tX, tY) = 0.93
accuracy(tX, tY) = 0.935
accuracy(tX, tY) = 0.93
accuracy(tX, tY) = 0.935
accuracy(tX, tY) = 0.923
accuracy(tX, tY) = 0.931
accuracy(tX, tY) = 0.93
accuracy(tX, tY) = 0.934
accurac